<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Onehot_A" data-toc-modified-id="Onehot_A-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Onehot_A</a></span></li><li><span><a href="#导入与分割数据" data-toc-modified-id="导入与分割数据-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>导入与分割数据</a></span></li><li><span><a href="#A_cat" data-toc-modified-id="A_cat-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>A_cat</a></span></li><li><span><a href="#A_hour" data-toc-modified-id="A_hour-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>A_hour</a></span></li><li><span><a href="#A_xgb" data-toc-modified-id="A_xgb-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>A_xgb</a></span></li><li><span><a href="#B_cat" data-toc-modified-id="B_cat-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>B_cat</a></span></li><li><span><a href="#A_his" data-toc-modified-id="A_his-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>A_his</a></span></li><li><span><a href="#结论" data-toc-modified-id="结论-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>结论</a></span></li><li><span><a href="#'C':-0.1" data-toc-modified-id="'C':-0.1-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>'C': 0.1</a></span></li></ul></div>

# Onehot_A

In [128]:

import numpy as np
import pandas as pd
import time
import gc
import os
import scipy.sparse as ss
#from  sklearn.cross_validation  import  train_test_split 
#import xgboost as xgb
#from sklearn.metrics import accuracy_score,log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,log_loss
from sklearn.model_selection import GridSearchCV

import numpy as np
import pandas as pd

import scipy.sparse as ss
from scipy.spatial.distance import jaccard, cosine

from sklearn.externals.joblib import dump, load

import utils



In [162]:
# 数据路径
dpath = utils.dpath
# 数据类型
data_types = utils.data_types
# 缓存数据路径
tmp_dpath = utils.tmp_dpath
# 距离计算公式
get_distance = utils.get_distance
to_0_1 = utils.normalization
to_cat = utils.label_encoder

# 导入users和events的index索引, 以及相关信息
users_index = load(tmp_dpath+'users_index.joblib.gz')
events_index = load(tmp_dpath+'events_index.joblib.gz')
all_user = set(users_index.keys())
all_event = set(events_index.keys())
num_users = len(users_index)
num_events = len(events_index)
user_event = load(tmp_dpath+'user_event.joblib.gz')
user_event_scores = load(tmp_dpath+'user_event_scores.joblib.gz')
data_df = load(tmp_dpath+'data_df.joblib.gz')

# 导入与分割数据

In [163]:
train = data_df.iloc[:15398,:].copy()
test = data_df.iloc[15398:,:].copy()

In [164]:
train[:5]

,event,interested,invited,not_interested,user,user_id,event_id,day,month,weekday,date,user_cf_dis,event_cf_dis,user_cf_reco,event_cf_reco,events_yes_num,events_all_num,users_freds_num
0,1918771225,0.0,0,0.0,3044012,1982,3396,0.033333,0.75,0.166667,0.689956,0.650327,0.684758,0.649747,0.589014,0.0008,0.0058,0.175499
1,1502284248,0.0,0,0.0,3044012,1982,1067,0.033333,0.75,0.166667,0.689956,0.666667,0.702021,0.666667,0.583333,0.0010,0.0140,0.175499
2,2529072432,1.0,0,0.0,3044012,1982,2302,0.033333,0.75,0.166667,0.689956,0.787475,0.823584,0.833625,0.000000,0.0212,0.4347,0.175499
3,3072478280,0.0,0,0.0,3044012,1982,7976,0.033333,0.75,0.166667,0.689956,0.666667,0.719680,0.666667,0.583333,0.0008,0.0023,0.175499
4,1390707377,0.0,0,0.0,3044012,1982,294,0.033333,0.75,0.166667,0.689956,0.692661,0.697196,0.696304,0.673430,0.0083,0.4017,0.175499


In [183]:
#导入数据
print('Load Data')
y_train = train['interested']

X_train = train.drop(['event', 'user', 'interested', 'not_interested','user_id','event_id','event_cf_reco'],axis=1)
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

Load Data


In [184]:
X_train.shape, y_train.shape, 

((15398, 11), (15398,))

In [185]:
#需要调优的参数
# 请尝试将L1正则和L2正则分开，并配合合适的优化求解算法（slover）
#tuned_parameters = {'penalty':['l1','l2'],
#                   'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
#                   }
solvers = ['liblinear','sag',]
Cs = [ 0.01, 0.1, 1, 10, 100, ]
tuned_parameters = dict(solver = solvers, C = Cs)

lr_penalty= LogisticRegression()
grid= GridSearchCV(lr_penalty, tuned_parameters,cv=5, scoring='neg_log_loss')


In [186]:
grid.fit(X_train,y_train)
grid.cv_results_

/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached w

{'mean_fit_time': array([ 0.02538733,  0.05645628,  0.0321032 ,  0.06278462,  0.04058223,
         0.09569955,  0.04730344,  0.29671464,  0.05554504,  0.4938724 ]),
 'mean_score_time': array([ 0.00268779,  0.0024713 ,  0.00257134,  0.00248218,  0.00256252,
         0.00247049,  0.00255146,  0.00256634,  0.00255241,  0.00281839]),
 'mean_test_score': array([-0.38219254, -0.29439633, -0.17282228, -0.12587519, -0.0802084 ,
        -0.07021833, -0.05098977, -0.05025531, -0.04425649, -0.04435049]),
 'mean_train_score': array([-0.38189618, -0.29422957, -0.17247561, -0.12560677, -0.07977265,
        -0.06958058, -0.05014644, -0.04921342, -0.04301522, -0.04307132]),
 'param_C': masked_array(data = [0.01 0.01 0.1 0.1 1 1 10 10 100 100],
              mask = [False False False False False False False False False False],
        fill_value = ?),
 'param_solver': masked_array(data = ['liblinear' 'sag' 'liblinear' 'sag' 'liblinear' 'sag' 'liblinear' 'sag'
  'liblinear' 'sag'],
              mask = 

In [187]:
print(grid.best_score_)
print(grid.best_params_)

-0.0442564885057
{'C': 100, 'solver': 'liblinear'}


In [188]:

c=100

#导入模型

lr= LogisticRegression( C=c, n_jobs=-1, solver='liblinear')

#开始训练
start_time = time.time()
print('training . . . ')
lr.fit(X_train, y_train, )
print('cost time:{0}'.format(int(time.time() - start_time)))

#保存模型
#joblib.dump(lr, model_path+'LR_sklearn.model')

#进行评价
train_preds = lr.predict_proba(X_train)[:,1]
train_predictions = np.around(train_preds)

train_accuracy = accuracy_score(y_train, train_predictions)
print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
train_log_loss = log_loss(y_train, train_preds)
print ("Train log_loss: " , train_log_loss)



training . . . 
cost time:0
Train Accuary: 98.48%
Train log_loss:  0.042981129193


/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


In [171]:
X_test = test.drop(['event', 'user', 'interested', 'not_interested','user_id','event_id'],axis=1)
X_test[:5]

,invited,day,month,weekday,date,user_cf_dis,event_cf_dis,user_cf_reco,event_cf_reco,events_yes_num,events_all_num,users_freds_num
15398,0,0.966667,0.875,0.666667,0.947598,0.0,0.0,0.0,0.0,0.0061,0.0107,0.089686
15399,0,0.966667,0.875,0.666667,0.947598,0.0,0.0,0.0,0.0,0.0018,0.0104,0.089686
15400,0,0.966667,0.875,0.666667,0.947598,0.0,0.0,0.0,0.0,0.0026,0.0085,0.089686
15401,0,0.966667,0.875,0.666667,0.947598,0.0,0.0,0.0,0.0,0.0031,0.0100,0.089686
15402,0,0.966667,0.875,0.666667,0.947598,0.0,0.0,0.0,0.0,0.0021,0.0076,0.089686


In [172]:
X_train.max()

invited            1.000000e+00
day                1.000000e+00
month              1.000000e+00
weekday            1.000000e+00
date               1.000000e+00
user_cf_dis        1.165430e+00
event_cf_dis       9.999168e-01
user_cf_reco       1.170313e+00
event_cf_reco      1.050840e+16
events_yes_num     1.000000e+00
events_all_num     1.000000e+00
users_freds_num    1.000000e+00
dtype: float64

In [173]:
X_test.max()

invited            1.000000
day                1.000000
month              1.000000
weekday            1.000000
date               1.000000
user_cf_dis        0.529412
event_cf_dis       0.000000
user_cf_reco       0.000000
event_cf_reco      0.000000
events_yes_num     0.980300
events_all_num     1.000000
users_freds_num    0.988993
dtype: float64

In [174]:
train_preds = lr.predict_proba(X_train)[:,1]

In [175]:
test_preds = lr.predict_proba(X_test)[:,1]

In [176]:
train_preds.max(), test_preds.max()

(0.88807236238952081, 0.5)

In [177]:
user_event = test.loc[:,['user','event']].copy()
user_event.index = test['user']
user_event['pred'] = test_preds
user_event[:5]

,user,event,pred
user,,,
1776192,1776192,2877501688,0.5
1776192,1776192,3025444328,0.5
1776192,1776192,4078218285,0.5
1776192,1776192,1024025121,0.5
1776192,1776192,2972428928,0.5


In [178]:
submission = pd.Series(index = set(user_event.index),)
for u in set(user_event.index):
    data_tmp = user_event.loc[u,:]
    event_tmp = pd.Series(data_tmp['pred'].values, index = data_tmp['event'].values)
    events = event_tmp.sort_values(ascending=False).index.values
    submission[u] = ' '.join(events.astype(np.str))

In [160]:
submission

2686965763    1335898508 940261225 3546861867 2701293208 111...
629450757     3655295959 955398943 2093319113 2605913428 252...
511678471     1138354613 2601678976 3932617564 2608367718 31...
3730114567    130281276 1628013431 2973911742 3400838699 569...
2678956041    834360181 3118576636 2235916835 4130552113 921...
4180064266    3380687062 3099686443 3297792743 2422997693 26...
4095465484    172445691 569616021 814858005 2130383503 17489...
2872854546    457253413 2980258101 2392886869 790687673 1516...
3270633490    673098017 1272961219 1269035551 2007279414 424...
666374166     2407779614 786960145 899115709 1944996876 2052...
1753157654    252715505 1536770720 4073937824 2974348707 410...
3329238167    401945277 1882674933 3136211868 3323634350 342...
1645299738    2007279414 3950589515 3048578499 2149464820 20...
3264469018    4074266568 4258255746 2350143315 389608207 169...
2371436573    1716825317 2529072432 656598905 794747940 4042...
1551265824           1536783222 28812988

In [179]:
for c,v in zip(X_train.columns, lr.coef_[0], ):
    print(c,':',v)

invited : -2.19059369648e-28
day : -2.1345154147e-27
month : -2.91679965293e-27
weekday : -1.79429083043e-27
date : -2.94998557343e-27
user_cf_dis : -2.08352524506e-27
event_cf_dis : -2.42733910994e-27
user_cf_reco : -1.93597448753e-27
event_cf_reco : 1.97099542318e-16
events_yes_num : -2.48893428346e-29
events_all_num : -6.25880926215e-28
users_freds_num : -8.29845380912e-28


In [76]:
submission.to_csv(tmp_dpath+'submission.csv')